In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
#計算準確度
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result

#
# v_xs: input
# v_ys: output
# keep_prob是保留概率，即我们要保留的结果所占比例
# 使输入tensor中某些元素變為0，其它没變0的元素變為原来的1/keep_prob大小！

In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)  #截斷常態分佈
    return tf.Variable(initial)
# stddev:標準差

In [5]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [6]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


In [7]:
def conv2d_layer1(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 4, 4, 1], padding='SAME')

# x:input
# W:weight
# padding: same、valid
# stride [1, x_movement, y_movement, 1]      
# Must have strides[0] = strides[3] = 1


In [8]:
def max_pool_overlap(x):
    return tf.nn.max_pool(x, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')
# stride [1, x_movement, y_movement, 1]


In [9]:
def norm(x):
    return tf.nn.lrn(x, depth_radius=2, bias=2.0, alpha=1e-4, beta=0.75)


In [10]:
# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, 784])/256   # 28x28
ys = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs, [-1, 28, 28, 1])    #[n_samples個數,圖長,圖寬,圖高]
print(x_image.shape)  # [n_samples, 28,28,1]


(?, 28, 28, 1)


In [11]:
## conv1 layer ##
W_conv1 = weight_variable([11,11,1,96])
b_conv1 = bias_variable([96])
r_conv1 = tf.nn.relu(conv2d_layer1(x_image, W_conv1) + b_conv1)
n_conv1 = norm(r_conv1)
n_pool1 = max_pool_overlap(n_conv1)


In [12]:
## split layer 1 to 2 tensor ##
split_n_pool1_up, split_n_pool1_down = tf.split(n_pool1, [48,48], 3)


In [13]:
## conv2 layer up ##
W_conv2_up = weight_variable([5,5,48,128])
b_conv2_up = bias_variable([128])
r_conv2_up = tf.nn.relu(conv2d(split_n_pool1_up, W_conv2_up) + b_conv2_up)
n_conv2_up = norm(r_conv2_up)
n_pool2_up = max_pool_overlap(n_conv2_up) 


In [14]:
## conv2 layer down ##
W_conv2_down = weight_variable([5,5,48,128])
b_conv2_down = bias_variable([128])
r_conv2_down = tf.nn.relu(conv2d(split_n_pool1_down, W_conv2_down) + b_conv2_down)
n_conv2_down = norm(r_conv2_down)
n_pool2_down = max_pool_overlap(n_conv2_down)


In [15]:
## merge layer 2 to 1 tensor ##
merge_layer2 = tf.concat([n_pool2_up, n_pool2_down], 3)


In [16]:
## conv3 layer ##
W_conv3 = weight_variable([3,3,256,384])
b_conv3 = bias_variable([384])
r_conv3 = tf.nn.relu(conv2d(merge_layer2, W_conv3) + b_conv3)

In [17]:
## split layer 3 to 2 tensor ##
split_n_pool3_up, split_n_pool3_down = tf.split(r_conv3, [192,192], 3)


In [18]:
## conv4 layer up ##
W_conv4_up = weight_variable([3,3,192,192])
b_conv4_up = bias_variable([192])
r_conv4_up = tf.nn.relu(conv2d(split_n_pool3_up, W_conv4_up) + b_conv4_up)


In [19]:
## conv4 layer down ##
W_conv4_down = weight_variable([3,3,192,192])
b_conv4_down = bias_variable([192])
r_conv4_down = tf.nn.relu(conv2d(split_n_pool3_down, W_conv4_down) + b_conv4_down)


In [20]:
## conv5 layer up ##
W_conv5_up = weight_variable([3,3,192,128])
b_conv5_up = bias_variable([128])
r_conv5_up = tf.nn.relu(conv2d(r_conv4_up, W_conv5_up) + b_conv5_up)
n_pool5_up = max_pool_overlap(r_conv5_up)


In [21]:
## conv5 layer down ##
W_conv5_down = weight_variable([3,3,192,128])
b_conv5_down = bias_variable([128])
r_conv5_down = tf.nn.relu(conv2d(r_conv4_down, W_conv5_down) + b_conv5_down)
n_pool5_down = max_pool_overlap(r_conv5_down)


In [22]:
## merge layer 5 to 1 tensor ##
merge_layer5 = tf.concat([n_pool5_up, n_pool5_down], 3)


In [23]:
# [n_samples, 7, 7, 64] ->> [n_samples, 7*7*64]
layer5_flat = tf.reshape(merge_layer5, [-1, 25600])


In [24]:
## fc1 layer ##
W_fc1 = weight_variable([25600, 4096])
b_fc1 = bias_variable([4096])

h_fc1 = tf.nn.relu(tf.matmul(layer5_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


In [25]:
## fc2 layer ##
W_fc2 = weight_variable([4096, 4096])
b_fc2 = bias_variable([4096])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)


In [27]:
## fc3 layer ##

#now using mnist label = 10
W_fc3 = weight_variable([4096, 10])
b_fc3 = bias_variable([10])

In [28]:
prediction = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)


In [29]:
# the error between prediction and real data
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
                                              reduction_indices=[1]))       # loss
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)
sess = tf.Session()

init = tf.global_variables_initializer()     # before 2017-03 use : init = tf.initialize_all_variables()
sess.run(init)


[<tf.Tensor 'gradients/split_1_grad/concat:0' shape=(?, 2, 2, 384) dtype=float32>, None, None]
[<tf.Tensor 'gradients/split_grad/concat:0' shape=(?, 4, 4, 96) dtype=float32>, None, None]


In [30]:
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 0.5})
    if i % 50 == 0:
        print(compute_accuracy(
            mnist.test.images[:1000], mnist.test.labels[:1000]))


ValueError: Dimensions must be equal, but are 10 and 1000 for 'Equal' (op: 'Equal') with input shapes: [10], [1000].